In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
# from my_parsers import download, parse
from tqdm import tqdm
from collections import Counter
# from brown_clustering import BigramCorpus, BrownClustering

pd.options.display.max_rows = 200
tqdm.pandas()

C:\Users\iltya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\iltya\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_TRAIN_RES_1 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')
#df_TRAIN_RES_2 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')
#df_TRAIN_RES_3 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')
#df_TRAIN_RES_4 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')
#df_TRAIN_RES_5 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')

#df_res = pd.concat([df_TRAIN_RES_1, df_TRAIN_RES_2, df_TRAIN_RES_3, df_TRAIN_RES_4, df_TRAIN_RES_5], axis=0)
df_res = df_TRAIN_RES_1.copy()
df_sal = pd.read_csv('vprod_train/TRAIN_SAL.csv')

C:\Users\iltya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,16,24,69) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# удаляем модифицированный столбец - он совпадает с точностью до регистра
df_TRAIN_RES_1.drop("achievements_modified", axis=1, inplace=True)

df_TRAIN_RES_1["achievements"].fillna("", inplace=True)
df_TRAIN_RES_1["achievements_len"] = [len(i) for i in df_TRAIN_RES_1["achievements"]]

df_TRAIN_RES_1.drop_duplicates(inplace=True)

In [55]:
df_TRAIN_RES_1[df_TRAIN_RES_1["achievements_len"] == 4770]

,achievements,company_name,demands,id_cv,job_title,achievements_isna,achievements_len
687466,Список реализованных объектов (проектов):1. ФГ...,"ЗАО РОСГЕОПРОЕКТ (изыскания, землеустройство, ...","- создание проектного подразделения ""с нуля"" п...",5f217020-f969-11e9-8b91-ef76bd2a03c1,руководитель,False,4770
687485,Список реализованных объектов (проектов):1. ФГ...,"ЗАО РОСГЕОПРОЕКТ (изыскания, землеустройство, ...","- создание проектного подразделения ""с нуля"" п...",4cf01980-f034-11e9-868a-ef76bd2a03c1,руководитель,False,4770
687494,Список реализованных объектов (проектов):1. ФГ...,"ЗАО РОСГЕОПРОЕКТ (изыскания, землеустройство, ...","- создание проектного подразделения ""с нуля"" п...",08c129f0-f969-11e9-8b91-ef76bd2a03c1,руководитель,False,4770
687508,Список реализованных объектов (проектов):1. ФГ...,"ЗАО РОСГЕОПРОЕКТ (изыскания, землеустройство, ...","- создание проектного подразделения ""с нуля"" п...",eb4a05d0-f969-11e9-8b91-ef76bd2a03c1,руководитель,False,4770
687511,Список реализованных объектов (проектов):1. ФГ...,"ЗАО РОСГЕОПРОЕКТ (изыскания, землеустройство, ...","- создание проектного подразделения ""с нуля"" п...",980d94e0-f969-11e9-8b91-ef76bd2a03c1,руководитель,False,4770


In [61]:
(df_TRAIN_RES_1["id_cv"] == "a8b302e8-6aba-11ef-acc9-cb26dff57dd7").sum()

0

In [65]:
df_TRAIN_RES_1["id_cv"].iloc[520]

'e387b9b9-2ea3-11e8-9855-0f468c90bfa7'

In [67]:
df_TRAIN_RES_1.iloc[-1]

achievements                                                          
company_name                                           ООО "Алкотрейд"
demands              Продажи алкогольной продукции. Контроль дебито...
id_cv                             631171a0-7f45-11ea-a3e6-037acc02728d
job_title                                                представитель
achievements_isna                                                 True
achievements_len                                                     0
Name: 1999999, dtype: object

In [100]:
df_TRAIN_RES_1["job_title"].isna().sum()

0

In [68]:
len(df_TRAIN_RES_1)

1664467

In [97]:
df_TRAIN_RES_1

,achievements,company_name,demands,id_cv,job_title,achievements_isna,achievements_len
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,True,0
1,,Администрация города Сердобска,Работа с реестром муниципальной собственности,52860148-26da-11e8-a184-9122a281f90e,инспектор,True,0
2,"Работы не закончены, т.к. нет настоящего желан...",январь 2016-настоящее времяООО «АРТЕЛЬ СТАРАТЕ...,Курирую работы по рудному направлению: соавтор...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,False,185
3,,"ОАО «Васильевский рудник», п.Мотыгино, Красноя...",ГРП. Документация керна разведочных скважин,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
4,,"ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",Поисково-разведочные работы на месторождении р...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
...,...,...,...,...,...,...,...
1999995,Мое резюме на hh.ru:https://hh.ru/resume/134eb...,Текта Групп,Мое резюме на hh.ru:https://hh.ru/resume/134eb...,0c919cc0-7e59-11ea-ba8b-ef76bd2a03c1,менеджер,False,80
1999996,,"ООО""Роста""",Сортировка заказов,653b46e0-8521-11ea-9fd7-b905beff6f7a,сортировщик,True,0
1999997,,ООО ВансладРитейл,Планировать и контролировать выполнение заказо...,d2925930-84cd-11ea-80d4-79a78a7da306,диспетчер,True,0
1999998,,"ООО ""Агроторг""",Продажи алкогольной продукции. Контроль дебито...,631171a0-7f45-11ea-a3e6-037acc02728d,представитель,True,0


In [101]:
t = df_TRAIN_RES_1[df_TRAIN_RES_1["id_cv"] == "e3c857b0-26a0-11e7-bcca-736ab11edb0c"]

In [4]:
used_classes = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False).iloc[:200].index.tolist()

In [5]:
df_TRAIN_RES_1 = df_TRAIN_RES_1[df_TRAIN_RES_1["job_title"].isin(used_classes)]

In [6]:
len(df_TRAIN_RES_1)

1594617

In [7]:
df_TRAIN_RES_1 = df_TRAIN_RES_1.iloc[:100_000]

In [8]:
df_TRAIN_RES_1["id_cv_0"] = [i.split("-")[0] for i in df_TRAIN_RES_1["id_cv"]]
df_TRAIN_RES_1["id_cv_1"] = [i.split("-")[1] for i in df_TRAIN_RES_1["id_cv"]]
df_TRAIN_RES_1["id_cv_2"] = [i.split("-")[2] for i in df_TRAIN_RES_1["id_cv"]]
df_TRAIN_RES_1["id_cv_3"] = [i.split("-")[3] for i in df_TRAIN_RES_1["id_cv"]]
df_TRAIN_RES_1["id_cv_4"] = [i.split("-")[4] for i in df_TRAIN_RES_1["id_cv"]]

In [9]:
df_TRAIN_RES_1 = df_TRAIN_RES_1.drop("id_cv", axis=1)

text_features = ["achievements", "company_name", "demands"]
cat_features = ["id_cv_0", "id_cv_1", "id_cv_2", "id_cv_3", "id_cv_4"]

In [10]:
for f in cat_features:
    print(f, len(df_TRAIN_RES_1[f].unique()))

id_cv_0 46538
id_cv_1 21582
id_cv_2 8
id_cv_3 1172
id_cv_4 29


In [11]:
df_TRAIN_RES_1.drop(["id_cv_0", "id_cv_1"], axis=1, inplace=True)
cat_features.remove("id_cv_0")
cat_features.remove("id_cv_1")

In [12]:
for text_feature in text_features:
    df_TRAIN_RES_1[text_feature].fillna("", inplace=True)

In [13]:
df_TRAIN_RES_1.drop(text_features, axis=1, inplace=True)
text_features = []

In [14]:
X, y = df_TRAIN_RES_1.drop("job_title", axis=1), df_TRAIN_RES_1["job_title"]

In [31]:
X.drop("achievements_len", axis=1, inplace=True)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1]:
text_features

NameError: name 'text_features' is not defined

In [33]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train, y_train, cat_features=cat_features, text_features=text_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features, text_features=text_features)

In [34]:
clf = CatBoostClassifier(task_type="GPU",  eval_metric="Accuracy")
clf.fit(train_pool, eval_set=test_pool, early_stopping_rounds=5)

Learning rate set to 0.163698
0:	learn: 0.1126375	test: 0.1163500	best: 0.1163500 (0)	total: 1.49s	remaining: 24m 51s
1:	learn: 0.1128125	test: 0.1165500	best: 0.1165500 (1)	total: 3.07s	remaining: 25m 33s
2:	learn: 0.1127125	test: 0.1165500	best: 0.1165500 (1)	total: 4.58s	remaining: 25m 22s
3:	learn: 0.1127750	test: 0.1165500	best: 0.1165500 (1)	total: 6.16s	remaining: 25m 34s
4:	learn: 0.1129000	test: 0.1166000	best: 0.1166000 (4)	total: 8.14s	remaining: 26m 59s
5:	learn: 0.1129250	test: 0.1166000	best: 0.1166000 (4)	total: 9.76s	remaining: 26m 57s
6:	learn: 0.1129500	test: 0.1166000	best: 0.1166000 (4)	total: 11.5s	remaining: 27m 6s
7:	learn: 0.1129750	test: 0.1166000	best: 0.1166000 (4)	total: 13.2s	remaining: 27m 12s
8:	learn: 0.1130125	test: 0.1166000	best: 0.1166000 (4)	total: 14.8s	remaining: 27m 13s
9:	learn: 0.1130500	test: 0.1166000	best: 0.1166000 (4)	total: 16.5s	remaining: 27m 17s
bestTest = 0.1166
bestIteration = 4
Shrink model to first 5 iterations.


In [35]:
clf.get_feature_importance(prettified=True)

,Feature Id,Importances
0,id_cv_2,70.551784
1,id_cv_4,29.448216
2,id_cv_3,0.000000


In [36]:
(y_test=="специалист").mean()

0.11555

In [122]:

for i in df_TRAIN_RES_1["id_cv_2"].unique():
    t = df_TRAIN_RES_1[df_TRAIN_RES_1[f"id_cv_{2}"] == i]
    print(i, t["job_title"].value_counts().iloc[:5])

11e8 специалист    36040
менеджер      21901
инженер       21629
бухгалтер     21562
оператор      12673
Name: job_title, dtype: int64
11e7 специалист    17802
инженер       12292
менеджер      10602
бухгалтер     10506
оператор       5774
Name: job_title, dtype: int64
11e6 специалист    16270
инженер       11136
менеджер      10471
бухгалтер      9584
оператор       4843
Name: job_title, dtype: int64
11e5 специалист     12469
инженер         9466
менеджер        9217
бухгалтер       7212
заместитель     3970
Name: job_title, dtype: int64
11eb водитель     51
продавец     36
бухгалтер    23
оператор     21
повар        16
Name: job_title, dtype: int64
11ea менеджер      56010
специалист    47867
бухгалтер     26733
продавец      24382
водитель      20690
Name: job_title, dtype: int64
11e9 специалист    34992
бухгалтер     21083
менеджер      20676
инженер       19412
оператор      15193
Name: job_title, dtype: int64
11e4 специалист     8
директор       5
инженер        5
заместитель   

специалист                 36040
менеджер                   21901
инженер                    21629
бухгалтер                  21562
оператор                   12673
                           ...  
помощник президента            1
парамедик                      1
зашивальщик мягкой тары        1
аэрографист                    1
гибщик                         1
Name: job_title, Length: 1097, dtype: int64

In [112]:
len(df_TRAIN_RES_1)

1664467

In [102]:
t

,achievements,company_name,demands,id_cv,job_title,achievements_isna,achievements_len
2,"Работы не закончены, т.к. нет настоящего желан...",январь 2016-настоящее времяООО «АРТЕЛЬ СТАРАТЕ...,Курирую работы по рудному направлению: соавтор...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,False,185
3,,"ОАО «Васильевский рудник», п.Мотыгино, Красноя...",ГРП. Документация керна разведочных скважин,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
4,,"ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",Поисково-разведочные работы на месторождении р...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
5,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
6,,ООО «АРТЕЛЬ СТАРАТЕЛЕЙ «СИНИНДА-1» п.Нижнеанга...,Детальные поиски и разведка с применением коло...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,True,0
7,Внедрил в практику работ компьютерные технолог...,ООО «АРТЕЛЬ СТАРАТЕЛЕЙ «СИНИНДА-1» п.Нижнеанга...,Проекты: Поисково-оценочные работы с применени...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,False,620
8,,Администрация Северо-Байкальского района п.Ниж...,"Разработка программ развития района, статистич...",e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,True,0
9,"Удалось нормализовать работу ЗАО ""Тоннель""",ОАО «Бамтоннельстрой» (дочернии предприятия ЗА...,Финансово-экономический контроль за работой до...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,False,42
10,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,True,0
11,Отчет по детальным поискам на Юго-западном фла...,СЕВЕРО-БАЙКАЛЬСКАЯ ГЕОЛОГОРАЗВЕДОЧНАЯ ЭКСПЕДИЦ...,Проведение поисковых и разведочных работ на Хо...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,False,159


In [ ]:
df_TRAIN_RES_1["com"]

In [ ]:
df_TRAIN_RES_1[df_TRAIN_RES_1["job_title"] == "преподаватель"]

In [94]:
t["demands"].iloc[9]

'Проведение занятий с обучающимися (бакалавриат, магистратура) по дисциплинам: Менеджмент| | Основы менеджмента и маркетинга| | Экономика предприятия| | Теория и практика управления. Осуществление обучения и воспитания обучающихся с учетом специфики преподаваемых предметов, проведение занятий в соответствии с расписанием и учебным планом в указанных помещениях. Обеспечение высокого уровня подготовки, соответствующий требованиям ФГОС ВО'

In [75]:
(t["demands"] == "педагогическая деятельность, ведение документации").sum()

0

In [76]:
t

,achievements,company_name,demands,id_cv,job_title,achievements_isna,achievements_len
60,,Благовещенский профессиональный лицей,"Преподавание литературы, общение со студентами",bb6ab9d0-283d-11e8-93a1-ef76bd2a03c1,преподаватель,True,0
306,Участие учеников на Российском соревновании Ro...,"ООО ""Восходящие технологии""",Обучение визуальному программируванию в среде ...,e6e85590-2c4d-11e8-b956-037acc02728d,преподаватель,False,52
1032,,"Международный славянский институт, Бишкекский ...",Преподавание дисциплин по клинической психолог...,92766740-2d10-11e8-b77d-736ab11edb0c,преподаватель,True,0
1034,,Кыргызский национальный университет,Преподавание дисциплин по клинической психолог...,92766740-2d10-11e8-b77d-736ab11edb0c,преподаватель,True,0
1077,,Саратовский педагогический колледж им. Гагарина,Преподавание,8fdb2fc9-313e-11e8-9855-0f468c90bfa7,преподаватель,True,0
...,...,...,...,...,...,...,...
1998685,,ККИ РУК,Преподавание информатики студентам колледжа,3fe01b10-7ff8-11ea-9939-736ab11edb0c,преподаватель,True,0
1999113,,Автономная некоммерческая организация высшего ...,"- организация и проведение учебной, воспитател...",f52313a0-84c2-11ea-a01c-b905beff6f7a,преподаватель,True,0
1999135,,Детский подростковый центр «Олимп»,Внешкольная деятельность. Обучение детей англи...,769caab0-848f-11ea-bdb4-1fdc17069750,преподаватель,True,0
1999136,,•МБУДО «Дом творчества Октябрьского района гор...,Внешкольная деятельность. Обучение детей англи...,769caab0-848f-11ea-bdb4-1fdc17069750,преподаватель,True,0


In [51]:
print(df_TRAIN_RES_1[df_TRAIN_RES_1["achievements_len"] == 4770]["achievements"].unique()[0])

Список реализованных объектов (проектов):1. ФГУП "Национальные рыбные ресурсы", Реконструкция объекта федеральной собственности морского терминала, предназначенного для комплексного обслуживания судов рыбопромыслового флота с объемом обработки грузов до 400 тыс. т в год., в Петропавловском морском порту, г. Петропавловск-Камчатский. Стоимость СМР более 5 млрд. руб.|2. Департамент развития новых территорий г. Москвы, Школа на 1100 мест по адресу: г. Москва, ТиНАО, поселение Сосенское, пос. Коммунарка, общая площадь объекта: 24 200 кв.м. Стоимость СМР более 1 млрд. руб.|3. ГУП "Мосгортранс", Объект: г. Москва, "Восстановление трамвайного движения на участке от госпиталя имени Н. Н. Бурденко до станции метро "Бауманская" |4. АО "ГУОВ": 4.1. "Общежитие на 800 мест, Военно-Космической академии имени А. Ф. Можайского, ул. Пионерская д. 59 литера А", общая площадь объекта 11 250 кв.м.|4.2. "Казарма на 800 мест и столовая на 1200 мест Военно-космической академии имени А. Ф. Можайского", по адр

In [40]:
sorted(df_TRAIN_RES_1["achievements_len"], reverse=True)

[13760,
 13760,
 13760,
 13710,
 11793,
 11793,
 11793,
 11793,
 11793,
 11793,
 11793,
 11793,
 11793,
 9814,
 9814,
 9814,
 9296,
 9204,
 9204,
 9204,
 8816,
 8750,
 8750,
 8461,
 8429,
 8097,
 8069,
 7996,
 7990,
 7936,
 7862,
 7858,
 7835,
 7835,
 7835,
 7835,
 7835,
 7487,
 7286,
 7284,
 7188,
 7055,
 7054,
 7010,
 6852,
 6852,
 6852,
 6852,
 6798,
 6729,
 6710,
 6710,
 6710,
 6710,
 6708,
 6686,
 6686,
 6686,
 6646,
 6583,
 6581,
 6484,
 6481,
 6481,
 6481,
 6301,
 6284,
 6266,
 6211,
 6205,
 6204,
 6091,
 6074,
 6017,
 6006,
 6006,
 5997,
 5969,
 5967,
 5967,
 5966,
 5960,
 5875,
 5863,
 5863,
 5863,
 5863,
 5863,
 5863,
 5863,
 5863,
 5861,
 5767,
 5707,
 5707,
 5683,
 5633,
 5607,
 5603,
 5594,
 5508,
 5487,
 5477,
 5477,
 5477,
 5466,
 5454,
 5407,
 5405,
 5402,
 5401,
 5398,
 5390,
 5386,
 5266,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5265,
 5247,
 5244,
 5243,
 5236,
 5231,
 5218,
 5205,
 5205,
 5202,
 5187,


In [23]:
t = df_TRAIN_RES_1[~df_TRAIN_RES_1["achievements"].isna()]

In [25]:
t[t["achievements"] != t["achievements_modified"]]

,achievements,achievements_modified,company_name,demands,id_cv,job_title
2181,не имею,нет,АО «Ликеро-водочный завод»,Ответственная,64157720-2de1-11e8-a3e9-e37b4be0b9ed,оператор
2362,НЕТ,нет,ООО.ЖЕЛЕЗНОДОРОЖНИК,Приемка локомотива.Содержание исправном состоянии,7d488ee6-2e62-11e8-a184-9122a281f90e,машинист
14061,достижений нет,нет,"ПОУ ""РУЦ"" РО ДОСААФ России Тюменской области",Разработка планирующей документации| Контроль ...,20af3a70-3584-11e8-8e6a-ef76bd2a03c1,заместитель
14666,Нет,нет,ООО Свод Интернешнл,Мытье посуды,e05dc680-3420-11e8-8b84-ef76bd2a03c1,стюард
14667,Нет,нет,ООО Свод Интернешнл,Мвтьё помкдв,e05dc680-3420-11e8-8b84-ef76bd2a03c1,стюард
...,...,...,...,...,...,...
1998677,Нет,нет,ООО Форт Пос Челябинск,Обучение персонала,7c528da0-83ac-11ea-a312-e736a3d3ed84,менеджер
1998680,Нет,нет,ООО ТД Нева,Продажа продуктов питания,7c528da0-83ac-11ea-a312-e736a3d3ed84,представитель
1998737,Нет,нет,ООО кубаньалкоопт,Прием товара,ec68ab20-8464-11ea-a79e-bfd13399602c,контролер
1998783,Нет,нет,Силовые машины,"Контроль изделий после механической обработки,...",e1a91c50-8320-11ea-b18f-03fd6b4dc11f,специалист


In [17]:
df_TRAIN_RES_1.isna().sum() / len(df_TRAIN_RES_1) * 100

achievements             84.13625
achievements_modified    84.13625
company_name              0.07885
demands                   0.79030
id_cv                     0.00000
job_title                 0.00000
dtype: float64

In [11]:
df_TRAIN_RES_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   achievements           object
 1   achievements_modified  object
 2   company_name           object
 3   demands                object
 4   id_cv                  object
 5   job_title              object
dtypes: object(6)
memory usage: 91.6+ MB


In [15]:
(df_TRAIN_RES_1.job_title.value_counts() > 1000).sum()

157

In [3]:
job_list = pd.read_csv("vprod_train/JOB_LIST.csv", encoding_errors="ignore")

In [4]:
job_list.dropna(inplace=True, axis=0)

In [5]:
def base_preproc(text):
    text = text.replace(",", " ").strip().lower()
    text = re.sub(' +', ' ', text)
    text = "".join([x for x in text if not x.isdigit()])
    return text

In [6]:
job_list["job_title"] = job_list["job_title"].map(base_preproc)

In [ ]:
my_counter = {}
for job in job_list["job_title"].tolist():
    for x in job.split():
        if x not in list(my_counter.keys())

In [ ]:
words = sum([x.split() for x in job_list["job_title"].tolist()], [])

In [ ]:
len(words)

In [107]:
bad_classes_df = df_res[df_res["id_cv"].isin(df_res[df_res["job_title"] == "специалист"]["id_cv"].unique())]

In [115]:
k = []

for ids, gr in tqdm(df_res.groupby("id_cv")):
    k.append(len(gr["company_name"].unique()))

100%|████████████████████████████████| 745438/745438 [00:15<00:00, 47155.67it/s]


In [134]:
df_sal.isna().sum()

id                                      0
academic_degree                    630830
accommodation_capability                0
accommodation_type                 597838
additional_premium                 549987
additional_requirements            400830
bonus_type                         549481
measure_type                       549778
busy_type                               0
career_perspective                      0
change_time                          5001
code_external_system               555416
code_profession                     14958
code_professional_sphere               14
company_code                            0
contact_person                        108
contact_source                     568063
data_ids                                0
date_create                             0
date_modify                             0
deleted                                 0
education                               0
education_speciality               515614
foreign_workers_capability        

In [138]:
from sage.spelling_correction import AvailableCorrectors
from sage.spelling_correction import RuM2M100ModelForSpellingCorrection, T5ModelForSpellingCorruption

text_ru = "Замтьте не я это предложил"
text_en = "Screw you kuys, I am going home. (c)"

corrector_fred = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_fredt5_large.value)
corrector_m2m = RuM2M100ModelForSpellingCorrection.from_pretrained(AvailableCorrectors.m2m100_1B.value)
corrector_en = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.ent5_large.value)

print(corrector_fred.correct(text_ru))
# ['Заметьте, не я это предложил.']

print(corrector_m2m.correct(text_ru))

ModuleNotFoundError: No module named 'sage'